# Send training request to Google AI Platform

In [2]:
BUCKET_NAME = 'gs://fma_bucket' #@param {type:"string"}
JOB_NAME = 'my_job_test3' #@param {type:"string"}
JOB_DIR = 'gs://fma_bucket/' + JOB_NAME + '/models' #@param {type:"string"}
REGION = 'us-central1' #@param {type:"string"}
IMAGE_URI='gcr.io/cloud-ml-public/training/pytorch-gpu.1-7' #@param {type:"string"}

In [53]:
!gsutil ls $BUCKET_NAME

gs://fma_bucket/fma_small/


## Test small job locally first

In [5]:
!gcloud ai-platform local train \
 --module-name trainer.train \
 --package-path ./src \
 --job-dir $JOB_DIR \
 -- \
 --num-epochs 3 \
 --batch-size 8 \
 --learning-rate 0.001 \
 --net "UNet"

    Cannot import tensorflow under path /opt/conda/bin/python. Using "chief" for cluster setting.
    If this is not intended, Please check if tensorflow is installed. Please also
    verify if the python path used is correct. If not, to change the python path:
    use `gcloud config set ml_engine/local_python $python_path`
    Eg: gcloud config set ml_engine/local_python /usr/bin/python3
    Cannot import tensorflow under path /opt/conda/bin/python. Using "chief" for cluster setting.
    If this is not intended, Please check if tensorflow is installed. Please also
    verify if the python path used is correct. If not, to change the python path:
    use `gcloud config set ml_engine/local_python $python_path`
    Eg: gcloud config set ml_engine/local_python /usr/bin/python3
Copying gs://fma_preprocessed_data/X_1.h5...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled b

## Run a small job

In [ ]:
!gcloud ai-platform jobs submit training $JOB_NAME \
 --package-path trainer/ \
 --module-name trainer.train \
 --master-image-uri $IMAGE_URI \
 --region $REGION \
 --scale-tier BASIC_GPU \
 --job-dir $JOB_DIR \
 -- \
 --num-epochs 100 \
 --batch-size 128 \
 --learning-rate 0.01 \
 --net "UNet"

# Stream the logs from the job
!gcloud ai-platform jobs stream-logs $JOB_NAME

Job [my_job_test3] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe my_job_test3

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs my_job_test3
jobId: my_job_test3
state: QUEUED
INFO	2021-05-10 21:46:44 +0000	service		Validating job requirements...
INFO	2021-05-10 21:46:44 +0000	service		Job creation request has been successfully validated.
INFO	2021-05-10 21:46:44 +0000	service		Waiting for job to be provisioned.
INFO	2021-05-10 21:46:44 +0000	service		Job my_job_test3 is queued.
INFO	2021-05-10 21:46:45 +0000	service		Waiting for training program to start.
INFO	2021-05-10 21:53:32 +0000	master-replica-0		
INFO	2021-05-10 21:53:32 +0000	master-replica-0		
INFO	2021-05-10 21:53:32 +0000	master-replica-0		
INFO	2021-05-10 21:53:32 +0000	master-replica-0		
INFO	2021-05-10 21:53:32 +0000	master-replica-0		
INFO	2021-05-10 21:57:22 +0000	master-replica-0		INFO

In [28]:
# Verify model was uploaded
!gsutil ls gs://fma_bucket

Removing gs://fma_bucket/pytorch-job-dir/packages/a6d40b7344cd12822e3d3505c0114b514db626c52b5bfac15bfcc09d418ff905/trainer-0.0.0.tar.gz#1620605583944796...
Removing gs://fma_bucket/pytorch-job-dir/packages/ec99a0eb6b19e8767257111fcf846b0d7128e855e43fa57f5d198fd689e5523d/trainer-0.0.0.tar.gz#1620605564255878...
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
